In [1]:
# Imports
import json
from pandas.io.json import json_normalize
import requests
import pyspark as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from textwrap import wrap
import folium
import plotly.graph_objects as go


In [2]:
# Methods
def unpack_column(dataFrame,col_name):
    newDf = dataFrame.from_records(dataFrame[col_name].dropna().tolist())
    return newDf

def unpack_and_destroy(dataFrame,column):
    temp = unpack_column(dataFrame,column)
    for col in temp.columns:
        dataFrame[col] = temp[col]
    #dataFrame = dataFrame.drop(columns=[column])
    return dataFrame

def sort_db_value(db,value,asc=False):
    return db.sort_values(by = value, ascending = asc)

def freq_array(freq_list):
    freq = []
    for i in freq_list:
        freq.append(i)
    return freq

def set_labels(freq, ax,horz_shift=.2,vert_shift=1000,w="bold",fsize=10,c="b"):
    for i, v in enumerate(freq):
        ax.text(i-horz_shift,v+vert_shift, str(round(v,2)), weight = w,fontsize=fsize,color = c)
        
def wrap_axis_labels(labels_list,wrap_length=13):
    labels_list = [ '\n'.join(wrap(l, wrap_length)) for l in labels_list]
    return labels_list

In [3]:
region1_df = pd.read_csv('Data/eo1.csv')
region2_df = pd.read_csv('Data/eo2.csv')
region3_df = pd.read_csv('Data/eo3.csv')

In [4]:
all_regions_df = pd.concat([region1_df,region2_df,region3_df])

In [5]:
all_regions_df.shape

(1711634, 28)

In [6]:
all_regions_df = all_regions_df[all_regions_df["DEDUCTIBILITY"] == 1]
all_regions_df.shape

(1418907, 28)

In [7]:
# all_regions_df = all_regions_df.drop(\
#     columns=['FILING_REQ_CD',"PF_FILING_REQ_CD","STATUS","NTEE_CD"])

In [17]:
# # State Count Bar Graph
# state_df_count = all_regions_df.groupby("STATE").size()
# states = list(state_df_count.index.values)
# freq = freq_array(state_df_count)

# fig,ax = plt.subplots(figsize = (40,20))
# ax.bar(states,freq, color = "b");
# ax.set_yticks(np.arange(0,np.max(state_df_count)+10000,10000));
# ax.set_title("Number of Organizations by State", weight = "bold", fontsize = 40);
# ax.set_xlabel("State", fontsize=30, weight = "bold");
# ax.set_ylabel("Frequency", fontsize=30, weight = "bold");

# set_labels(state_df_count,ax,.4,1500,c="k");

In [16]:
# state_df_count = all_regions_df.groupby("STATE").size().reset_index(name="Frequency").sort_values(by="Frequency",ascending=False)
# state_df_count.iloc[:10]

In [18]:
# # State Count Bar Graph (Top 10)
# state_df_count = all_regions_df.groupby("STATE").size().reset_index(name="Frequency").sort_values(by="Frequency",ascending=False)
# state_df_count = state_df_count.iloc[:10]

# states = state_df_count['STATE']
# freq = freq_array(state_df_count['Frequency'])
# fig,ax = plt.subplots(figsize = (20,10))
# ax.bar(states,freq, color = "b");

# ax.set_yticks(np.arange(0,np.max(freq)+20000,20000));
# ax.set_title("Number of Organizations by State (Top 10)", weight = "bold", fontsize = 30);
# ax.set_xlabel("State", fontsize=25, weight = "bold");
# ax.set_ylabel("Frequency", fontsize=25, weight = "bold");
# ax.set_yticklabels(np.arange(0,np.max(freq)+20000,20000), weight="bold", fontsize =20) ;
# ax.set_xticklabels(states, weight="bold",fontsize=20);

# set_labels(freq,ax,.3,1500,c="k",fsize=20);

# plt.savefig("Num_Orgs_Top_10_States")

In [11]:
# my_USA_map = "Data/us-states.json"
# map = folium.Map(location=[40, -100], zoom_start=4)

In [20]:
# state_df = all_regions_df.groupby("STATE").size().reset_index(name="Count")

In [13]:
# map.choropleth(geo_data=my_USA_map, data=state_df,
#              columns=['STATE', 'Count'],
#              key_on='feature.id',
#              fill_color='YlGn', fill_opacity=0.8, line_opacity=0.2,
#              legend_name='Charity Count', hover_text=state_df["STATE"])
# map

In [24]:
fig = go.Figure(data=go.Choropleth(
    locations = state_df['STATE'],
    z=state_df['Count'],
    locationmode='USA-states',
    colorscale='Blues',
    autocolorscale=False,
    text=state_df['STATE'], # hover text
    marker_line_color='black', # line markers between states
    colorbar_title="Number of Charities"
))

fig.update_layout(
    title_text='Number of US Charitable Organizations by State - IRS<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

In [ ]:
all_regions_df.sort_values(by="INCOME_AMT",ascending=False).head()

In [ ]:
total_assets = all_regions_df.groupby("STATE")["ASSET_AMT"].agg("sum").reset_index(name="Sum")
total_assets.head()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = total_assets['STATE'],
    z=total_assets['Sum'],
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=total_assets['STATE'], # hover text
    marker_line_color='black', # line markers between states
    colorbar_title="Sum of Assets"
))

fig.update_layout(
    title_text='Sum of US Charitable Organizations Assets by State<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

In [ ]:
def heatmap_USA_by_state(dataFrame,state_col_name,zoom_col_name,title,bar_title,action="sum"):
    temp_df = dataFrame.groupby(state_col_name)[zoom_col_name].agg(action).reset_index(name=action)
    
    fig = go.Figure(data=go.Choropleth(
    locations = temp_df[state_col_name],
    z=temp_df[action],
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,

    text = temp_df[state_col_name],

    marker_line_color='black', # line markers between states
    colorbar_title=bar_title
    ))

    fig.update_layout(
        title_text=title+'<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=go.layout.geo.Projection(type = 'albers usa'),
            showlakes=True, # lakes
            lakecolor='rgb(255, 255, 255)'),
    )

    fig.show()


In [ ]:
#Consider Adding Charity Names in Hover
heatmap_USA_by_state(
    all_regions_df,'STATE',"INCOME_AMT","Max Income Charities By State","Max Income","max"
)

In [ ]:
heatmap_USA_by_state(
    all_regions_df,'STATE',"INCOME_AMT","Total Income Charities By State","Total Income","sum"
)

In [ ]:
all_regions_df.head()